# The Benchmarking Interlude

To get the total time taken to run multiple calls to a function with arbitrary positional arguments, the following first- cut function might suffice:

In [ ]:
from timer0 import timer

In [ ]:
timer(pow, 2, 4) # Time to call pow(2, 4) 1000 times

In [ ]:
pow?

In [ ]:
timer(str.upper, 'spam') # Time to call 'spam'.upper() 1000 times

Though simple, this timer is also fairly limited, and deliberately exhibits some classic mistakes in both function design and benchmarking. Among these, it:

* Doesn’t support keyword arguments in the tested function call
* Hardcodes the repetitions count
* Only gives total time, which might fluctuate on some heavily loaded machines

In [ ]:
import timer

In [ ]:
timer.total(1000, pow, 2, 1000)

In [ ]:
timer.total(1000, str.upper, 'spam')

In [ ]:
timer.bestof(1000, str.upper, 'spam')

In [ ]:
timer.bestof(1000, pow, 2, 1000000)[0]

In [ ]:
timer.bestof(50, timer.total, 1000, str.upper, 'spam')

In [ ]:
timer.bestoftotal(50, 1000, str.upper, 'spam')

### timeit module

In [ ]:
import timeit

In [ ]:
timeit.repeat?

In [ ]:
min(timeit.repeat(stmt="[x ** 2 for x in range(1000)]", number=100, repeat=5))

In [ ]:
timeit.timeit(stmt='[x ** 2 for x in range(1000)]', number=1000) # Total time

In [ ]:
def testcase():
    y = [x ** 2 for x in range(1000)] # Callable objects or code strings

In [ ]:
min(timeit.repeat(stmt=testcase, number=1000, repeat=3))

# Modules: The Big Picture

In concrete terms, modules typically correspond to Python program files. Each file is a module, and modules import other modules to use the names they define.

**import**:
Lets a client (importer) fetch a module as a whole

**from**:
Allows clients to fetch particular names from a module

**imp.reload**:
Provides a way to reload a module’s code without stopping Python

### Why Use Modules?

In short, modules provide an easy way to organize components into a system by serving as self-contained packages of variables known as namespaces. 

More specifically, modules have at least three roles

* Code reuse

* System namespace partitioning

* Implementing shared services or data

### Python Program Architecture

In practice, programs usually involve more than just one file. Python fosters a modular program structure that groups functionality into coherent and reusable units.

Below figure sketches the structure of a Python program composed of three files: a.py, b.py, and c.py. The file a.py is chosen to be the top-level file; it will be a simple text file of statements, which is executed from top to bottom when launched. The files b.py and c.py are modules; they are simple text files of statements as well, but they are not usually launched directly. Instead, modules are normally imported by other files that wish to use the tools the modules define.

![alt text](./figures/python_architecture.png)

In Python, imports are really runtime operations that perform three distinct steps the first time a program imports a given file:

1. Find the module’s file.

2. Compile it to byte code (if needed).

3. Run the module’s code to build the objects it defines.

### The Module Search Path

If you want to be able to import userdefined files across directory boundaries, though, you will need to know how the search path works in order to customize it.

Roughly, Python’s module search path is composed of the concatenation of these major components, some of which are preset for you and some of which you can tailor to tell Python where to look:

1. The home directory of the program
2. PYTHONPATH directories (if set)
3. Standard library directories
4. The contents of any .pth files (if present)
5. The site-packages home of third-party extensions

The first and third elements of the search path are defined automatically. Because Python searches the concatenation of these components from first to last, though, the second and fourth elements can be used to extend the path to include your own source code directories. 

If you want to see how the module search path is truly configured on your machine, you can always inspect the path as Python knows it by printing the built-in ``sys.path``.

In [ ]:
import sys
print(sys.path)

Keep in mind that filename extensions (e.g., .py) are omitted from import statements intentionally. Python chooses the first file it can find on the search path that matches the imported name. In fact, imports are the point of interface to a host of external components—source code, multiple flavors of byte code, compiled extensions, and more. Python automatically selects any type that matches a module’s name.

# Module Coding Basics

**Module Creation:** To define a module, simply use your text editor to type some Python code into a text file, and save it with a “.py” extension; any such file is automatically considered a Python module. 

**Module File names:** should end in a .py suffix if you plan to import them. 

**The import statement:**

In [ ]:
import module1  # Get module as a whole (one or more)
module1.printer('Hello world!') # Qualify to get names

**The from statement:**

In [ ]:
from module1 import printer  # copy out a variable (one or more)
printer('Hello world!')

**The from * Statement:**

In [1]:
from module1 import *  # copy out all variables
printer('Hello world!')

5 6 7
Hello world!


In [2]:
A

5

In [3]:
B

6

In [4]:
module1.A

NameError: name 'module1' is not defined

**Imports Happen Only Once**

In [5]:
import simple # First import: loads and runs file's code

hello


In [6]:
simple.spam # Assignment makes an attribute

1

Second and later imports don’t rerun the module’s code; they just fetch the already created module object from Python’s internal modules table. Thus, the variable spam is not reinitialized:

In [7]:
simple.spam = 2  # change attribute in module

In [8]:
import simple  # just fetches already loaded module

In [9]:
simple.spam # code wasn't rerun: attribute unchanged

2

**Changing mutables in modules**

In [10]:
from small import x, y  # Copy two names out
x = 42                  # changes local x only
y[0] = 42               # changes shared mutable in place

Here, x is not a shared mutable object, but y is. The names y in the importer and the importee both reference the same list object, so changing it from one place changes it in the other:

In [11]:
import small    # Get module name
small.x  # Small's x is not my x

1

In [12]:
small.y # But we share a changed mutable

[42, 2]

**import and from Equivalence**

is equivalent to this staement sequence:

**When import is required**

the from statement will fail - you can only have one assignment to the name in your scope:

An import will work here, though, because including the name of the enclosing module makes the two names unique:

In [13]:
# O.py
import M  # Get the whole modules, not their names
import N
M.func()     # We can call both names now
N.func()     # The module names make them unique

func of M
func of N


Another way out of this dilemma is using the as extension:

In [14]:
from M import func as mfunc   # Rename uniquely with 'as'
from N import func as nfunc
mfunc(); nfunc()             # Calls one or the other

func of M
func of N


**Module Namespaces**

In [15]:
import module2

starting to load...
done loading.


The first time this module is imported (or run as a program), Python executes its statements from top to bottom.

In [16]:
import module2

Once the module is loaded, its scope becomes an attribute namespace in the module object we get back from ``import``. 

In [17]:
module2.name

42

In [18]:
module2.__name__

'module2'

In [19]:
module2.func

<function module2.func()>

In [20]:
module2.klass

module2.klass

We can access a module’s namespace dictionary through the module’s __dict__ attribute.

In [21]:
list(module2.__dict__.keys())

['__name__',
 '__doc__',
 '__package__',
 '__loader__',
 '__spec__',
 '__file__',
 '__cached__',
 '__builtins__',
 'sys',
 'name',
 'func',
 'klass']

Python also adds some names in the module’s namespace for us; for instance, __file__ gives the name of the file the module was loaded from, and __name__ gives its name as known to importers (without the .py extension and directory path).

**Namespace Nesting**

In [22]:
run mod1.py

2 3
1 2 3


In [23]:
run mod1.py

1 2 3


**Reloading Modules**

As we’ve seen, a module’s code is run only once per process by default. To force a module’s code to be reloaded and rerun, you need to ask Python to do so explicitly by calling the reload built-in function.

In [24]:
import changer

In [25]:
changer.printer()

First version


In [26]:
import changer

In [27]:
changer.printer() # No effect: uses loaded module

First version


In [28]:
from imp import reload

In [29]:
reload(changer) # Forces new code to load/run

<module 'changer' from '/Users/sergulaydore/Documents/courses/EE 551A Fall 2018/lectures/changer.py'>

In [30]:
changer.printer() # Runs the new version now

After editing


# Module Packages

In addition to a module name, an import can name a directory path. A directory of Python code is said to be a package, so such imports are known as *package imports*.

At a base level, package imports are straightforward:

These two import statements imply a directory structure that looks something like this:

dir0/dir1/dir2/mod.py

The container directory dir0 needs to be added to your module search path unless it’s
the home directory of the top-level file.

**Package ``__init__.py`` files**

For a directory structure such as this:

and an import statement of the form:

the following rules apply:

* dir1 and dir2 both must contain an ``__init__.py`` file.

* dir0, the container, does not require an ``__init__.py`` file; this file will simply be ignored if present.

* dir0, not dir0\dir1, must be listed on the module search path sys.path.

The net effect is that this example’s directory structure should be as follows:

The ``__init__.py`` files can contain Python code, just like normal module files. Their names are special because their code is run automatically the first time a Python program imports a directory, and thus serves primarily as a hook for performing initialization steps required by the package. 

**Package Import Example**:

``import`` statements run each directory’s initialization file the first time that directory is traversed, as Python descends the path; ``print`` statements are included here to trace their execution:

In [1]:
import dir1.dir2.mod # First imports run init files

dir1 init
dir2 init
in mod.py


In [2]:
import dir1.dir2.mod # Later imports do not

Just like module files, an already imported directory may be passed to reload to force reexecution of that single item. 

In [3]:
from imp import reload

In [5]:
reload(dir1.dir2)

dir2 init second attempt


<module 'dir1.dir2' from '/Users/sergulaydore/Documents/courses/EE 551A Fall 2018/lectures/dir1/dir2/__init__.py'>

Once imported, the path in your import statement becomes a nested object path in your script.

In [6]:
dir1

<module 'dir1' from '/Users/sergulaydore/Documents/courses/EE 551A Fall 2018/lectures/dir1/__init__.py'>

In [7]:
dir1.dir2

<module 'dir1.dir2' from '/Users/sergulaydore/Documents/courses/EE 551A Fall 2018/lectures/dir1/dir2/__init__.py'>

In [8]:
dir1.dir2.mod

<module 'dir1.dir2.mod' from '/Users/sergulaydore/Documents/courses/EE 551A Fall 2018/lectures/dir1/dir2/mod.py'>

In [9]:
dir1.x

1

In [10]:
dir1.dir2.y

2

In [11]:
dir1.dir2.mod.z

3

**from Versus import with Packages**

``import`` statements can be somewhat inconvenient to use with packages, because you may have to retype the paths frequently in your program.

In [12]:
from dir1.dir2 import mod # Code path here only

In [13]:
mod.z # Don't repeat path

3

In [14]:
from dir1.dir2.mod import z

In [15]:
import dir1.dir2.mod as mod # Use shorter nam

In [16]:
from dir1.dir2.mod import z as modz

The only time package imports are actually required is to resolve ambiguities that may arise when multiple programs with same-named files are installed on a single machine.

Suppose we have:

So far, there’s no problem: both systems can coexist and run on the same computer.

However, suppose that after you’ve installed these two programs on your machine, you decide that you’d like to use some of the code in each of the utilities.py files in a system of your own.

To make this work at all, you’ll have to set the module search path to include the directories containing the utilities.py files. But which directory do you put first in the path—system1 or system2?

This is the issue that packages actually fix. 

Now, add just the common root directory to your search path.

**Package Relative Imports**

Rather than listing package import paths, imports within the package can be relative to the package.

**Why relative imports?**

Consider the following package directory:

An additional leading dot performs the relative import starting from the parent of the current package.

# Advanced Module Topics

As a summary, below figure sketches the environment in which modules operate:

![alt text](./figures/module_environment.png)

As a special case, you can prefix names with a single underscore (e.g., _X) to prevent them from being copied out when a client imports a module’s names with a ``from *``

In [19]:
from unders import * # Load non _X names only

In [20]:
a, c

(1, 3)

In [21]:
_b

NameError: name '_b' is not defined

In [25]:
import unders # But other importers get every name

In [26]:
unders._b

2

Alternatively, you can achieve a hiding effect similar to the ``_X`` naming convention by assigning a list of variable name strings to the variable ``__all__`` at the top level of the module.

In [27]:
from alls import *  # Load __all__ names onl

In [28]:
a, _c

(1, 3)

In [29]:
b

NameError: name 'b' is not defined

In [30]:
from alls import a, b, _c, _d   # But other importers get every name

In [31]:
a, b, _c, _d

(1, 2, 3, 4)

In [32]:
import alls

In [33]:
alls.a, alls.b, alls._c, alls._d

(1, 2, 3, 4)

**Enabling Future Language Features: __future__**

In [34]:
from __future__ import division

**Mixed Usage Modes: ``__name__`` and ``__main__``**

Each module has a built-in attribute called ``__name__``, which Python creates and assigns automatically as follows:

* If the file is being run as a top-level program file, ``__name__`` is set to the string "``__main__``" when it starts.

* If the file is being imported instead, ``__name__`` is set to the module’s name as known by its clients.

In [35]:
import runme

In [40]:
cd ..

/Users/sergulaydore/Documents/courses/EE 551A Fall 2018/lectures


In [41]:
run runme.py

It's Christmas in Heaven...


**Docstrings: Module Documentation at Work**

In [42]:
import permute

In [43]:
help(permute)

Help on module permute:

NAME
    permute

FUNCTIONS
    permute1(seq)
    
    permute2(seq)

FILE
    /Users/sergulaydore/Documents/courses/EE 551A Fall 2018/lectures/permute.py




In [44]:
reload(runme)

<module 'runme' from '/Users/sergulaydore/Documents/courses/EE 551A Fall 2018/lectures/runme.py'>

In [45]:
help(runme)

Help on module runme:

NAME
    runme - This is a docstring

FUNCTIONS
    tester()

FILE
    /Users/sergulaydore/Documents/courses/EE 551A Fall 2018/lectures/runme.py




**Changing the Module Search Path**

In [46]:
import sys

In [47]:
sys.path

['',
 '/Users/sergulaydore/Documents/courses/EE 551A Fall 2018/lectures',
 '/Users/sergulaydore/.virtualenvs/EE551/lib/python37.zip',
 '/Users/sergulaydore/.virtualenvs/EE551/lib/python3.7',
 '/Users/sergulaydore/.virtualenvs/EE551/lib/python3.7/lib-dynload',
 '/usr/local/Cellar/python/3.7.0/Frameworks/Python.framework/Versions/3.7/lib/python3.7',
 '/Users/sergulaydore/.virtualenvs/EE551/lib/python3.7/site-packages',
 '/Users/sergulaydore/Documents/courses/EE 551A Fall 2018/github/homework2-sergulaydore/src',
 '/Users/sergulaydore/.virtualenvs/EE551/lib/python3.7/site-packages/IPython/extensions',
 '/Users/sergulaydore/.ipython']

In [48]:
sys.path.append('/Users/sergulaydore/sourcedir') # Extend module search path

In [49]:
sys.path

['',
 '/Users/sergulaydore/Documents/courses/EE 551A Fall 2018/lectures',
 '/Users/sergulaydore/.virtualenvs/EE551/lib/python37.zip',
 '/Users/sergulaydore/.virtualenvs/EE551/lib/python3.7',
 '/Users/sergulaydore/.virtualenvs/EE551/lib/python3.7/lib-dynload',
 '/usr/local/Cellar/python/3.7.0/Frameworks/Python.framework/Versions/3.7/lib/python3.7',
 '/Users/sergulaydore/.virtualenvs/EE551/lib/python3.7/site-packages',
 '/Users/sergulaydore/Documents/courses/EE 551A Fall 2018/github/homework2-sergulaydore/src',
 '/Users/sergulaydore/.virtualenvs/EE551/lib/python3.7/site-packages/IPython/extensions',
 '/Users/sergulaydore/.ipython',
 '/Users/sergulaydore/sourcedir']